In [1]:
import sys
sys.path.append("..")

# backends
from qiskit.providers.fake_provider import Fake27QPulseV1,Fake5QV1,Fake20QV1


# poly_sabre
from isl_sabre.poly_circuit_preprocess import *
from isl_sabre.poly_circuit_utils import *
from isl_sabre.poly_sabre import POLY_SABRE


# qiskit
from qiskit import QuantumCircuit, transpile
from qiskit.transpiler.passes import SabreLayout, SabreSwap
from qiskit.transpiler import PassManager,CouplingMap
from qiskit.converters import circuit_to_dag
from qiskit.transpiler.layout import Layout

In [2]:
edges = Fake27QPulseV1().configuration().coupling_map

In [3]:
file_path = "../benchmarks/polyhedral/queko-bss-16qbt/16QBT_100CYC_QSE_1.json"
data = json_file_to_isl(file_path)


In [4]:
def run_sabre(data):
    circuit = QuantumCircuit.from_qasm_str(data["qasm_code"])
    coupling_map = CouplingMap(edges)

    # Step 1: Run SabreLayout to determine the initial layout
    sabre_layout = SabreLayout(coupling_map, seed=21)
    layout_pass_manager = PassManager(sabre_layout)
    layout_applied_circuit = layout_pass_manager.run(circuit)

    # Step 2: Run SabreSwap separately to insert swap gates
    sabre_swap = SabreSwap(coupling_map, seed=21, heuristic="decay", trials=1)
    swap_pass_manager = PassManager(sabre_swap)
    optimized_circuit = swap_pass_manager.run(layout_applied_circuit)

    # Count the number of swap gates
    swap_count = optimized_circuit.count_ops().get("swap", 0)
    return swap_count,optimized_circuit

In [5]:
swap_count,optimized_circuit = run_sabre(data)

seed : 21 swap_trials: 6 layout_trials: 6
start routing --> sabre_routing
end routing --> sabre_routing


In [6]:
optimized_circuit.count_ops()

OrderedDict([('x', 816), ('cx', 320), ('swap', 104)])

In [7]:
#dag = circuit_to_dag(optimized_circuit)
#dag.draw()

In [8]:
#optimized_circuit.draw(output='mpl',scale=0.5)

In [9]:
data = json_file_to_isl(file_path)
poly_sabre = POLY_SABRE(edges,data)

In [10]:
poly_sabre.run(huristic_method="decay",verbose=1)

seed : 21 swap_trials: 6 layout_trials: 6


Executing Gates: 100%|██████████| 320/320 [00:38<00:00,  8.28it/s]


116

In [11]:
print(swap_count)

104


In [12]:
poly_sabre.run(verbose=1)

seed : 21 swap_trials: 6 layout_trials: 6


Executing Gates: 100%|██████████| 320/320 [00:19<00:00, 16.38it/s]


117

In [14]:
poly_sabre.run(huristic_method="poly-paths",verbose=1)

seed : 21 swap_trials: 6 layout_trials: 6


Executing Gates: 100%|██████████| 320/320 [00:10<00:00, 31.63it/s]


116